In [1]:
%load_ext jupyter_black

import os
from glob import glob
import pandas as pd
from wxlab.forecast import Grib


galwem_data = pd.Series(sorted(glob(os.path.join("data", "galwem", "*.GR2"))), name="GALWEM")
galwem_times: pd.DataFrame = galwem_data.str.extract(r"FH.(?P<forecast_hour>\d{3})_DF__(?P<valid_time>\d{8})")
galwem_data.index = pd.to_datetime(galwem_times["valid_time"]) + pd.to_timedelta(
    galwem_times["forecast_hour"].astype(int), unit="h"
)
galwem_data

/home/leaver2000/wxpy/wxlab/probsevere.py:20: UserWarning: registration of accessor <class 'wxlab.probsevere.GeoAccessor'> under name 'geo' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  class GeoAccessor:


2022-05-20 00:00:00    data/galwem/PS.557WW_SC.U_DI.C_GP.GALWEM-GD_GR...
2022-05-20 03:00:00    data/galwem/PS.557WW_SC.U_DI.C_GP.GALWEM-GD_GR...
2022-05-20 06:00:00    data/galwem/PS.557WW_SC.U_DI.C_GP.GALWEM-GD_GR...
Name: GALWEM, dtype: object

In [16]:
grib = Grib(galwem_data)
grib

In [20]:
# the dataset is 4 demensional ( T, X, Y, Z )
ds = grib.to_dataset()
ds

<xarray.Dataset>
Dimensions:    (validTime: 3, hPa: 13, lat: 141, lon: 281)
Coordinates:
  * validTime  (validTime) int64 1653004800000000000 ... 1653026400000000000
  * lat        (lat) float32 20.0 20.25 20.5 20.75 ... 54.25 54.5 54.75 55.0
  * lon        (lon) float32 230.0 230.2 230.5 230.8 ... 299.2 299.5 299.8 300.0
  * hPa        (hPa) float32 3e+03 5e+03 7e+03 1e+04 ... 7e+04 8.5e+04 1e+05
Data variables:
    temp       (validTime, hPa, lat, lon) float32 218.1 218.1 ... 274.4 275.2
    u_wind     (validTime, hPa, lat, lon) float32 -9.2 -9.13 -9.13 ... 4.08 4.62
    v_wind     (validTime, hPa, lat, lon) float32 0.31 0.16 0.07 ... 3.1 2.53

In [21]:
# when called to a dataframe the Z axis is moved to the second axis of dataframe
idx: slice = pd.IndexSlice
df = grib.to_dataframe()
df[idx[3000, "temp"]]

validTime            lat   lon   
1653004800000000000  20.0  230.00    218.069992
                           230.25    218.129990
                           230.50    218.220001
                           230.75    218.199997
                           231.00    218.059998
                                        ...    
1653026400000000000  55.0  299.00    224.259995
                           299.25    224.299988
                           299.50    224.349991
                           299.75    224.449997
                           300.00    224.580002
Name: (3000.0, temp), Length: 118863, dtype: float32

In [22]:
# with pyarrow the data can be turned into a table
table = grib.to_table()
table

pyarrow.Table
temp: float
u_wind: float
v_wind: float
validTime: int64
lat: double
lon: double
hPa: double
----
temp: [[218.06999,207.84999,199.87,202.09,209.4,...,237.04999,248.26999,265.43,268.47,275.25]]
u_wind: [[-9.2,-6.2799997,-4.3399997,6.8799996,24.769997,...,7.240002,7.27,8.03,9.889999,4.62]]
v_wind: [[0.31,-1.43,2.1299999,13.25,19.359999,...,3.4499998,1.1899999,-1.88,-5.46,2.53]]
validTime: [[1653004800000000000,1653004800000000000,1653004800000000000,1653004800000000000,1653004800000000000,...,1653026400000000000,1653026400000000000,1653026400000000000,1653026400000000000,1653026400000000000]]
lat: [[20,20,20,20,20,...,55,55,55,55,55]]
lon: [[230,230,230,230,230,...,300,300,300,300,300]]
hPa: [[3000,5000,7000,10000,15000,...,40000,50000,70000,85000,100000]]

In [23]:
# that table can be saved as a parquet file which can maintain the context of the multi index
grib.to_parquet("data/GALWEM.parquet")

In [24]:
# that parquet can be read back into Grib
grib2 = Grib("data/GALWEM.parquet")
grib2

In [25]:
# loading the grib from the parquet took 0.6 seconds versus the from the raw grib data which took almost 30 seconds
ds2 = grib2.to_dataset()
ds2

<xarray.Dataset>
Dimensions:    (validTime: 3, lat: 141, lon: 281, hPa: 13)
Coordinates:
  * validTime  (validTime) int64 1653004800000000000 ... 1653026400000000000
  * lat        (lat) float64 20.0 20.25 20.5 20.75 ... 54.25 54.5 54.75 55.0
  * lon        (lon) float64 230.0 230.2 230.5 230.8 ... 299.2 299.5 299.8 300.0
  * hPa        (hPa) float64 3e+03 5e+03 7e+03 1e+04 ... 7e+04 8.5e+04 1e+05
Data variables:
    temp       (validTime, lat, lon, hPa) float32 218.1 207.8 ... 268.5 275.2
    u_wind     (validTime, lat, lon, hPa) float32 -9.2 -6.28 -4.34 ... 9.89 4.62
    v_wind     (validTime, lat, lon, hPa) float32 0.31 -1.43 2.13 ... -5.46 2.53

In [19]:
grib.to_dataframe()

hPa                                3000.0      5000.0      7000.0    \
elements                               temp        temp        temp   
validTime           lat  lon                                          
1653004800000000000 20.0 230.00  218.069992  207.849991  199.869995   
                         230.25  218.129990  207.739990  199.839996   
                         230.50  218.220001  207.629990  199.819992   
                         230.75  218.199997  207.529999  199.879990   
                         231.00  218.059998  207.440002  199.879990   
...                                     ...         ...         ...   
1653026400000000000 55.0 299.00  224.259995  221.699997  221.879990   
                         299.25  224.299988  221.720001  221.750000   
                         299.50  224.349991  221.720001  221.639999   
                         299.75  224.449997  221.709991  221.559998   
                         300.00  224.580002  221.669998  221.470001   

hPa                                10000.0     15000.0     20000.0   \
elements                               temp        temp        temp   
validTime           lat  lon                                          
1653004800000000000 20.0 230.00  202.089996  209.399994  217.589996   
                         230.25  202.019989  209.389999  217.440002   
                         230.50  201.959991  209.259995  217.289993   
                         230.75  201.899994  209.069992  217.139999   
                         231.00  201.869995  208.889999  217.259995   
...                                     ...         ...         ...   
1653026400000000000 55.0 299.00  221.019989  226.190002  226.529999   
                         299.25  221.099991  226.000000  226.589996   
                         299.50  221.139999  225.839996  226.549988   
                         299.75  221.220001  225.669998  226.549988   
                         300.00  221.319992  225.419998  226.669998   

hPa                                25000.0     30000.0     40000.0   \
elements                               temp        temp        temp   
validTime           lat  lon                                          
1653004800000000000 20.0 230.00  229.019989  239.429993  252.839996   
                         230.25  229.080002  239.479996  253.039993   
                         230.50  229.059998  239.519989  253.279999   
                         230.75  228.929993  239.619995  253.509995   
                         231.00  228.860001  239.669998  253.569992   
...                                     ...         ...         ...   
1653026400000000000 55.0 299.00  224.220001  223.690002  236.720001   
                         299.25  224.129990  223.940002  236.739990   
                         299.50  224.049988  224.139999  236.759995   
                         299.75  223.899994  224.159988  236.860001   
                         300.00  223.610001  224.159988  237.049988   

hPa                                50000.0   ...   10000.0    15000.0   \
elements                               temp  ...     v_wind     v_wind   
validTime           lat  lon                 ...                         
1653004800000000000 20.0 230.00  264.070007  ...  13.250000  19.359999   
                         230.25  264.149994  ...  13.090000  19.580000   
                         230.50  264.119995  ...  12.840000  20.020000   
                         230.75  264.089996  ...  12.599999  20.459999   
                         231.00  264.209991  ...  12.360000  20.740000   
...                                     ...  ...        ...        ...   
1653026400000000000 55.0 299.00  248.189987  ...   3.380000   4.000000   
                         299.25  248.139999  ...   3.500000   4.050000   
                         299.50  248.080002  ...   3.590000   4.130000   
                         299.75  248.110001  ...   3.640000   4.220000   
                         300.00  248.269989  ...   3.